# Introduction / example

Suppose you can produce goods in different factories and sell them on different markets. Each factory has its own production capacity and each market has its own demand, which we may know after elaborating a survey. As the distances of the markets and the factories are different, we are looking for a transport route which minimizes the total costs.

The problem can be visualized as follows

![TransportProblem](transport_problem1.drawio.svg)


Reference: [@scipbook, introduction:Transport problem]

## Learnings

Again we assume that the reader is familiar with basic concepts of optimization, but we are writting the blog in a way that the unexperienced readers see the benefit of these techniques, while considering them as blakc boxes.

- SCIP `multidic`
- get slack variable values, sensitivity analysis, shadow prices, ... (all synonyms)

# Algebraic formulation 

The given problem can be described by the following abstract model:

## Sets

- $I$ customer set
- $J$ factory set

## Variables

- $x_{i,j}$ amount of goods transported from factory $j$ to customer $i$

## Parameter

- $c_{i,j}$ transportation costs from factory $i$ to customer $j$
- $d_i$ demand of customer $i$
- $m_j$ production capacity of factory $j$

## LP-Model

$$
\begin{array}{llll}
\min & \sum_{i,j} c_{i,j} x_{i,j} & & \\
s.t. & \sum_j x_{i,j}             & = d_i & \forall i\\
     & \sum_i x_{i,j}             & \leq m_j & \forall j\\
     & x_{i,j}                    & \geq 0 & \forall i,j
\end{array}
$$

In [ ]:
import pyscipopt as scip

In [ ]:
# demand
I, d = scip.multidict({1:80, 2:270, 3:250, 4:160, 5:180})
# capacities
J, M = scip.multidict({1:500, 2:500, 3:500})

In [ ]:
# transport costs
c = {(1,1):4,    (1,2):6,    (1,3):9,
     (2,1):5,    (2,2):4,    (2,3):7,
     (3,1):6,    (3,2):3,    (3,3):3,
     (4,1):8,    (4,2):5,    (4,3):3,
     (5,1):10,   (5,2):8,    (5,3):4,
     }

In [ ]:
m = scip.Model()   

# variables
x = {}
for i in I:
    for j in J:
        x[i,j] = m.addVar(vtype = 'C', name = 'x(%s,%s)' % (i,j))
    
# constraints
for i in I:
    m.addCons(scip.quicksum( x[i,j] for j in J if (i,j) in x) == d[i],
              name = 'CustomerDemand(%s)' % i)
for j in J:
    m.addCons(scip.quicksum( x[i,j] for i in I if (i,j) in x) <= M[j],
              name = 'FactoryCapacity(%s)' % j)
    
# objective
m.setObjective(scip.quicksum(c[i,j]*x[i,j] for (i,j) in x),
               sense = 'minimize')   

m.optimize()    

In [ ]:
print("optimal value:", m.getObjVal())
epsilon = 1.e-6
for (i,j) in x:
    if m.getVal(x[i,j]) > epsilon:
        print('send %10s goods from factory %2s to customer %2s' % (m.getVal(x[i,j]), j, i))

optimal value: 3350.0
send       80.0 goods from factory  1 to customer  1
send      270.0 goods from factory  2 to customer  2
send      230.0 goods from factory  2 to customer  3
send       20.0 goods from factory  3 to customer  3
send      160.0 goods from factory  3 to customer  4
send      180.0 goods from factory  3 to customer  5


# Duals, shadow prices, sensitivity analysis ...

A sensitivity analysis tells us how optimal solution and optimal value may change when we change the data used in the model. Since data may not always be considered as totally accurate, such analysis can be very helpful to the decision makers.

## Example continued

Suppose you believe your factory capacity got tight and you are considering an expansion and you ask yourself the following questions:

- What kind of costs can be reduced by expanding each factory?
- What is the additional profit you can make if you get additional orders from each customer?

In order to investigate whether or not a factory should be expanded, we look at the values of the slack variables of the capacity constraints:

$$
\sum_i x_{i,j}\leq M_j
$$

We recall that $x_{i,j}$ denotes the amount of goods transported from factory $j$ to market $i$ and $M_j$ denotes the capacity of factory $j$.
